In [ ]:
## Save & Load

In [1]:
import os
from glob import glob

import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets

## Hyper parameter Tunning

In [2]:
num_epochs = 1
batch_size = 32
learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs = inputs, outputs = net)
model.summary()

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0     

## Data Process

In [4]:
## (1) train, test 데이터 읽어오기
print(os.listdir('../dataset/cifar/'))

train_paths = glob('../dataset/cifar/train/*.png')[:1000]
test_paths = glob('../dataset/cifar/test/*.png')[:1000]

print(len(train_paths), len(test_paths))

## (2) label name 반환
def get_class_name(path):
    return path.split('_')[-1].replace('.png','')

## (3) label name 고유값 list - class_names
train_labels = [get_class_name(path) for path in train_paths] # 50000개
class_names = np.unique(train_labels) # 해당 class 개로 줄여짐

# (4) label 반환 onehot encoding 된 형태로 or argmax 로 해당 index
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) 

# (5) 최종 image, label 반환
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32)/.255 # rescale
    
    label = get_label(path)
    return image, label

# (6) image preprocess (랜덤 Generator)
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE # 작동하는 Network가 스스로 설정하고 Dataset을 잘 불러올 수 있게 결정하라는 것


# train dataset 뽑기
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

# test set 데이터 뽑기
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()
      

['labels.txt', 'test', 'train']
1000 1000


## Training

In [5]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_data = test_dataset,
    validation_steps = validation_steps,
    epochs = num_epochs,
)

Instructions for updating:
Please use Model.fit, which supports generators.
31/31 [==============================] - 4s 141ms/step - loss: 38.8377 - accuracy: 0.1023 - val_loss: 2.3001 - val_accuracy: 0.1210


## Saving Model
- 모델 구조는 json 으로 저장 가능

In [6]:
save_path = 'my_model.h5'
model.save(save_path, include_optimizer=True) # optimizer 정보들 (compile 정보들)

## Load Model 분석하기

In [7]:
import h5py
model_file = h5py.File('my_model.h5','r+')
print(type(model_file))
print(model_file.keys())
print(model_file['model_weights'].keys())


<class 'h5py._hl.files.File'>
<KeysViewHDF5 ['model_weights', 'optimizer_weights']>
<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>


In [8]:
print(model_file['model_weights']['conv2d']['conv2d']['bias:0'])
print(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

print(model_file['model_weights']['conv2d']['conv2d']['bias:0'].shape)
print(model_file['model_weights']['conv2d']['conv2d']['kernel:0'].shape)

<HDF5 dataset "bias:0": shape (32,), type "<f4">
<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">
(32,)
(3, 3, 3, 32)


In [10]:
# 가중치 저장 파일들도 나올수 있음
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])[0]

array([[[-0.11628272,  0.04443438, -0.1051142 ,  0.09131569,
         -0.04325047, -0.07531124, -0.02744057, -0.08719209,
         -0.03929242, -0.13524082,  0.05178919,  0.07031562,
          0.08606918, -0.11791819,  0.03716299, -0.10904998,
         -0.09573398,  0.0043032 ,  0.1360523 ,  0.01598894,
         -0.04175979, -0.02842781,  0.11513054,  0.04651206,
         -0.05695517,  0.02731065, -0.07074099,  0.11237109,
          0.06783808,  0.12595841, -0.12178364,  0.04286348],
        [ 0.01431291, -0.10566497,  0.05810326,  0.06199744,
         -0.10325361,  0.08371942, -0.1255439 , -0.11343787,
          0.00311637,  0.01732994,  0.09915306,  0.03831234,
         -0.04876991,  0.11916008, -0.01433842,  0.0801892 ,
         -0.10447582,  0.08245142,  0.07485044,  0.01187762,
         -0.03805762,  0.04243518, -0.03199923, -0.09041011,
          0.05651764, -0.04712895, -0.07127952, -0.06949459,
          0.07123994,  0.11279991, -0.01579679, -0.05093192],
        [-0.06934155, 